Containis function for feature extraction. Takes a directory and a boolean indicating wheather to split the data or not. For cross validation use the data directory (labeled data) and split = true. Run all the way to the cell that prints out accuracy score. For testing model on the unlabled data in test folder (those that are named sample001, etc), run the cells below the accuracy score cell will give you a .csv file ready for submission on kaggle. 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy.io import wavfile
import re
import seaborn as sns
import random
from sklearn import svm
import librosa
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re


In [5]:
# test data and test data directory 

data_dir = "/content/drive/MyDrive/ELEC378_FinalProject/data/data"

test_dir = "/content/drive/MyDrive/ELEC378_FinalProject/test/test"



In [14]:
def FeatureExtraction (dir, split=True): 
 
  '''
  Function for feature extraction. Change this section to change what features we are using. 

  input: a directory of the data. Will split to test and train, a boolean: if split is true, will extract features and labels and store them in 4 arrays:
  train_data, train_label, test_data, test_label. if split is false, function will extract all data provided in the directory and output 2 arrays with values 2 arrays that are empty. 
  Use labeled data and split = false will give you  (train_data, train_label); use unlabled data and split = false will give you (test_data, test_label).

  output: data matrix and lables for both test and train. 

  '''
  audio_files = [f for f in os.listdir(dir) if f.endswith(".wav")]
  
  train_files, test_files = train_test_split(audio_files, test_size=0.2)

  train_data = []
  train_label= []
  test_data = []
  test_label = []  
  
  
  for file_name in audio_files:

      # Load audio
      file_path = os.path.join(dir, file_name)
      raw_audio, sr = librosa.load(file_path)

      # if audio is too short, append zeros after it. 
      if np.shape(raw_audio)[0] < 80000: 
          padded_audio = np.pad(raw_audio, [(0, 80000 - np.shape(raw_audio)[0])], mode='constant')
      else:
          padded_audio = raw_audio

      # now slice so that we guarantee that each array has the same length
      audio = padded_audio[20000:80000]

      # feature extraction

      mfccs = librosa.feature.mfcc(y=audio, sr = 22050, n_mfcc = 20)
      #chroma_stft = librosa.feature.chroma_stft(y = audio, sr=22050)
      #mel_spec = librosa.feature.melspectrogram(y = audio, sr = 22050, n_mels = 5)
      #gfccs = librosa.feature.gfcc(y, sr=sr, n_mfcc=20)

      # Concatenate
      features = np.concatenate([mfccs.flatten()])
      
      label = file_name.split(".")[0]

      if label[:-3] == "sample":
        test_data.append(features)
        test_label.append(label)

      else:
        if split: 
          if file_name in train_files:
            train_data.append(features)
            train_label.append(label[:-3])

          elif file_name in test_files: 
            test_data.append(features)
            test_label.append(label[:-3])

        elif not split: 
          
          train_data.append(features)
          train_label.append(label[:-3])
  

  train_data = np.array(train_data)
  train_label = np.array(train_label)
  test_data = np.array(test_data)
  test_label = np.array(test_label) 

  
  return train_data, train_label, test_data, test_label



In [15]:
X_train, y_train, X_test, y_test = FeatureExtraction(data_dir, split = True)
print(np.shape(y_train))

(900,)


In [16]:
np.shape(y_test)

(226,)

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


In [17]:

clf = make_pipeline(StandardScaler(), SVC(kernel = "linear"))
clf.fit(X_train, y_train)

y_predicted= clf.predict(X_test)

accuracy = accuracy_score(y_test, y_predicted)

print(accuracy)

0.584070796460177


Below is for unlabeled test data and importing a csv


In [22]:
# extract all labeled data
X_train, y_train, dummy1, dummy2 = FeatureExtraction(data_dir, split = False)
# extract all unlabeled data
dummy1, dummy2, X_test, y_test = FeatureExtraction(test_dir, split = False)

Training and testing models 


In [23]:
clf = make_pipeline(StandardScaler(), SVC(kernel = "linear"))
clf.fit(X_train, y_train)

y_predicted= clf.predict(X_test)

In [26]:
# this cell is to export a .csv file to submit (filename & predicted label) 

import csv

with open('prediction.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['filename', 'label'])
    for i in range(len(y_test)):
      writer.writerow([y_test[i], y_predicted[i]])
file.close()

